In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [119]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
kniga = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 'Книга13.xlsx']

In [7]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [10:37<00:00, 57.91s/it]


In [8]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [9]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [10]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [11]:
kniga['Код склада'] = kniga['Код склада'].astype(str)
skladi = kniga['Код склада'].unique().tolist()

In [120]:
stock.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки'],
      dtype='object')

In [121]:
stock.columns = ['code', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [122]:
stock = stock.query('code in @skladi')

In [124]:
stock.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [125]:
result = stock.drop_duplicates(subset=['Код товара'])

In [126]:
tovs = result['ID Сайта'].dropna().tolist()

In [ ]:
images['article'] = images['article'].astype(str)

In [ ]:
images = images.query('article in @tovs')

In [19]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [20]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [130]:
result = result.merge(images, how='outer', on='ID Сайта')

In [131]:
result = result.drop(columns = ['Остаток'])

In [132]:
df = stock

In [133]:
stock = df.groupby(by = 'Код товара', as_index=False)['Остаток'].sum()

In [134]:
result = result.merge(stock, how='left', on='Код товара')

In [135]:
sale.columns = ['Код товара', 'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06']

In [136]:
result = result.merge(sale, how='left', on='Код товара')

In [137]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
       'Опубликован', 'Ссылка на изображение', 'Остаток',
       'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06'],
      dtype='object')

In [138]:
result = result.drop_duplicates(subset=['Код товара'])

In [139]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [146]:
clear_weight = df.groupby(by = 'Код товара', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['Код товара', 'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес мин'] / clear_weight['Чистый вес макс'] - 1) * -1

In [147]:
clear_weight

,Код товара,Чистый вес макс,Чистый вес медиана,Чистый вес мин,"Чистый вес размах, %"
0,ТОВ0115302,6.010,5.825,5.820,0.031614
1,ТОВ0120225,2.500,2.500,2.500,-0.000000
2,ТОВ0120668,11.590,11.590,11.590,-0.000000
3,ТОВ0121528,2.610,2.605,2.600,0.003831
4,ТОВ0121690,4.640,4.635,4.630,0.002155
...,...,...,...,...,...
125009,ТОВ1334829,0.577,0.572,0.567,0.017331
125010,ТОВ1334830,0.479,0.479,0.469,0.020877
125011,ТОВ1334831,0.549,0.539,0.539,0.018215
125012,ТОВ1335481,0.790,0.765,0.720,0.088608


In [148]:
result = result.merge(clear_weight, how='left', on='Код товара')

In [149]:
price_birka = df.groupby(by = 'Код товара', as_index=False)['Цена изделия на бирке'].agg({'min', 'median', 'max'})
price_birka.columns = ['Код товара', 'Цена изделия на бирке макс', 'Цена изделия на бирке медиана', 'Цена изделия на бирке мин']

In [150]:
price_birka

,Код товара,Цена изделия на бирке макс,Цена изделия на бирке медиана,Цена изделия на бирке мин
0,ТОВ0115302,74524.0,72230.0,72168.0
1,ТОВ0120225,38375.0,38375.0,38375.0
2,ТОВ0120668,293331.0,293331.0,293331.0
3,ТОВ0121528,14161.0,14136.5,14112.0
4,ТОВ0121690,57536.0,39185.5,20835.0
...,...,...,...,...
125009,ТОВ1334829,49673.0,49292.5,48912.0
125010,ТОВ1334830,42847.0,42366.0,41542.0
125011,ТОВ1334831,44432.0,43672.0,43227.0
125012,ТОВ1335481,18881.0,18283.5,17208.0


In [151]:
result = result.merge(price_birka, how='left', on='Код товара')

In [152]:
base_discount = df.groupby(by = 'Код товара', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['Код товара', 'Базовая скидка макс', 'Базовая скидка мин']

In [153]:
base_discount

,Код товара,Базовая скидка макс,Базовая скидка мин
0,ТОВ0115302,21.0,16.0
1,ТОВ0120225,0.0,0.0
2,ТОВ0120668,46.0,46.0
3,ТОВ0121528,0.0,0.0
4,ТОВ0121690,0.0,0.0
...,...,...,...
125009,ТОВ1334829,59.0,59.0
125010,ТОВ1334830,62.0,62.0
125011,ТОВ1334831,59.0,59.0
125012,ТОВ1335481,35.0,35.0


In [154]:
result = result.merge(base_discount, how='left', on='Код товара')

In [156]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии']/100) * 0.8 * 0.6
)

In [157]:
price_skidki = df.groupby(by = 'Код товара', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['Код товара', 'Цена со скидками макс', 'Цена со скидками медиана', 'Цена со скидками мин']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками мин'] / price_skidki['Цена со скидками макс'] - 1) * -1

In [159]:
result = result.merge(price_skidki, how='left', on='Код товара')

In [161]:
selfprice = df.groupby(by = 'Код товара', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['Код товара', 'Себестоимость макс', 'Себестоимость медиана', 'Себестоимость мин']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость мин'] / selfprice['Себестоимость макс'] - 1) * -1

In [162]:
result = result.merge(selfprice, how='left', on='Код товара')

In [163]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [164]:
markup = df.groupby(by = 'Код товара', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['Код товара', 'Наценка макс', 'Наценка медиана', 'Наценка мин']

In [166]:
result = result.merge(markup, how='left', on='Код товара')

In [169]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

TypeError: list indices must be integers or slices, not str

In [60]:
zapret = zapret['Артикул сайта'].tolist()

In [170]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [171]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.17.xlsx')

In [172]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [173]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1']

In [174]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [175]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [176]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [177]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [178]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [179]:
ecom = ecom.query('article in @tovs')

In [180]:
ecom.columns = ['ID Сайта', 'size', 'Цена со скидками Ecom']

In [181]:
result = result[result['ID Сайта'] != '']

In [182]:
result['ID Сайта'] = result['ID Сайта'].astype(int).astype(str)
result['Размер изделия'] = result['Размер изделия'].astype(str)

In [183]:
ecom['size'] = ecom['size'].fillna(0.0)
ecom['size'] = ecom['size'].astype(str)

In [184]:
ecom['merge'] = ecom['ID Сайта'] + ecom['size']

In [185]:
result['merge'] = result['ID Сайта'] + result['Размер изделия']

In [186]:
ecom = ecom.drop(columns=['ID Сайта', 'size'])

In [187]:
result = result.merge(ecom, how='outer', on='merge')

In [188]:
result = result.drop(columns=['merge'])

In [189]:
result = result.dropna(subset=['Код товара'])

In [190]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [191]:
summa_ost = df.groupby(by ='Код товара', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [192]:
result = result.merge(summa_ost, how='left', on='Код товара')

In [193]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [194]:
price_per_gramm = df.groupby(by = 'Код товара', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['Код товара', 'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин']

In [195]:
price_per_gramm

,Код товара,Цена за грамм макс,Цена за грамм медиана,Цена за грамм мин
0,ТОВ0115302,5947.615670,5947.580377,5448.302696
1,ТОВ0120225,4500.000000,4500.000000,4500.000000
2,ТОВ0120668,6560.085867,6560.085867,6560.085867
3,ТОВ0121528,5183.846154,5182.919246,5181.992337
4,ТОВ0121690,4500.000000,4500.000000,4500.000000
...,...,...,...,...
125009,ТОВ1334829,16976.863492,16959.529146,16942.194801
125010,ТОВ1334830,16315.851357,16156.206397,16132.689770
125011,ТОВ1334831,15945.546568,15927.536612,15783.067904
125012,ТОВ1335481,7456.800000,7456.800000,7456.800000


In [196]:
result = result.merge(price_per_gramm, how='left', on='Код товара')

In [197]:
result.to_csv('result.csv', index=False)

In [198]:
zapret_sk = df.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [199]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [200]:
yes = yes.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [201]:
yes.columns = ['Код товара', 'Количество с запрещенной скидкой']

In [202]:
no.columns = ['Код товара', 'Количество с разрешенной скидкой']

In [203]:
yes = yes.merge(no, how='outer', on='Код товара')

In [204]:
result = result.merge(yes, how='left', on='Код товара')

In [205]:
result = result.drop_duplicates(subset=['Код товара'])

In [206]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

In [207]:
df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)

In [208]:
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

In [209]:
data = df.groupby(by = 'Код товара', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

In [210]:
# df = df.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (c 01.01.2023)',
#     'Новая закупка (с 01.01.2023)'])

In [211]:
test = result[['Код товара', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %']]

In [212]:
test[test['Код товара'] == 'ТОВ1051309']

,Код товара,Цена со скидками макс,Цена со скидками медиана,Цена со скидками мин,"Цена со скидками размах, %"
68593,ТОВ1051309,8643.2064,7935.3576,2177.712,0.748044


In [213]:
data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Нет информации о дате'
        )
    )
)

In [214]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])

In [215]:
result = result.merge(data, how='left', on='Код товара')

In [216]:
result = result.drop(columns=['Дата закупки_x'])

In [220]:
result.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
    'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
    'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-11.06',
    'Сумма продаж 14.03-11.06', 'Чистый вес макс', 'Чистый вес медиана',
    'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Дата закупки']

In [224]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [225]:
result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [226]:
result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [227]:
result['Цена за грамм размах, %'] = (result['Цена за грамм мин'] / result['Цена за грамм макс'] - 1) * -1

In [228]:
result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [229]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

In [230]:
result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [231]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-11.06',
       'Сумма продаж 14.03-11.06', 'Чистый вес макс', 'Чистый вес медиана',
       'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
       'Цена со скидками медиана', 'Цена со скидками мин',
       'Цена со скидками размах, %', 'Себестои

In [232]:
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке мин'] / result['Цена изделия на бирке макс'] - 1) * -1

In [233]:
result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [235]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена Розн., за шт', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки'])

In [243]:
result = result[['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
    'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Опубликован', 'Ссылка на изображение', 'Остаток',
    'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06',
    'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
    'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин', 'Цена изделия на бирке размах, %',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин', 'Цена за грамм размах, %',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Размах по весу', 'Размах по себестоимости',
    'Размах по цене со скидками', 'Размах по цене за грамм', 'Размах цена изделия на бирке', 'Скидка разрешена', 'Дата закупки']]

In [245]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)